In [1]:
import sys
import numpy as np
import scipy
import subprocess
from natsort import natsorted
import itertools
import pandas as pd
import torch
sys.path.append("../utls")
sys.path.append("../utls")
sys.path.append("../.")
sys.path.append("../models")
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import torchvision
import torchvision.transforms
from sklearn.preprocessing import StandardScaler, LabelEncoder
from utls.preprocessing import *
from utls.postprocessing import get_dataset, generate_hist_df
from InterfaceDeclaration import LPBFInterface
from models.MLUtls import fade_in_out, standardize_tensor, getKFoldCrossValidationIndexes, train_log, transform_ft, dataset_by_cross_validation, labels_by_classes, get_current_fold_and_hist, LPBFDataset
from models.MLModels import SVMModel, CNN_Base_1D_Model, ResNet15_1D_Model


alphabet = list(string.ascii_lowercase)

project_name = ["MuSIC", "MaPS", "MuSIC_EXP1"]
if os.name == "posix":
    data_dir = subprocess.getoutput("echo $DATADIR")
elif os.name == "nt":
    data_dir = subprocess.getoutput("echo %datadir%")
music_dir = os.path.join(data_dir, "MuSIC")
if not os.path.exists(music_dir):
    project_name[0] = "2024-MUSIC"
daq_dir = os.path.join(data_dir, *project_name, "Acoustic Monitoring")
lmq_dir = os.path.join(data_dir, *project_name, "LMQ Monitoring")
del music_dir

sampling_rate_daq: int = int(1.25 * 1e6)
sampling_rate_lmq: int = int(0.1 * 1e6)
tdms_daq_list = natsorted(
    [i for i in os.listdir(daq_dir) if i.split(".")[-1] == "tdms"]
)
bin_lmq_list = natsorted([i for i in os.listdir(lmq_dir) if i.split(".")[-1] == "bin"])
lmq_channel_name = [
    "Vector ID",
    "meltpooldiode",
    "X Coordinate",
    "Y Coordinate",
    "Laser power",
    "Spare",
    "Laser diode",
    "Varioscan(focal length)",
]
process_regime = [
    [0,    59, "Base"  ], 
    [60,  129, "KH"    ], 
    [130, 199, "Normal"], 
    [200, 269, "RLoF"  ], 
    [269, 339, "LoF"   ] 
]

dataset,sc_power,le_direction,le_speed,le_region = get_dataset()

In [2]:
max_epochs=50
his_dir = os.path.join(os.pardir,"lfs","weights/hist")
snap_dir = os.path.join(os.pardir,"lfs","weights/")
model_name= ["CNN"]
acoustic_type =['ae','mic'] 
context_type = []
# context_type = ['energy']
output_type = ['direction']
folds = 10

df = generate_hist_df(his_dir,model_name,acoustic_type,context_type,output_type,folds,max_epochs)
df['Input Type'] = df['Input type'].str.replace('+', '\n')

In [3]:
def make_model(model_name, input_type, output_type, time_series_length=5888):
    meta_data_size = len(input_type.split("+"))-1
    # time_series_length = 5888
    # print(f"Input type: {input_type}")
    # print(f"Output type: {output_type}")
    # print(f"Lenght of time series data: {time_series_length}")
    # print(f"Lenght of context data: {meta_data_size}")
    # print(f"Model name: {model_name}")

    num_classes = 1
    if output_type == "regime":
        num_classes = 4
    if output_type == "defect":
        num_classes = 2
    if output_type == "direction":
        num_classes = 5
    if output_type == "position":
        num_classes = 5

    if model_name == "SVM":
        model = SVMModel(time_series_length+meta_data_size,num_classes=num_classes).double()
    if model_name == "CNN":
        model = CNN_Base_1D_Model(time_series_length=time_series_length, meta_data_size=meta_data_size,num_classes=num_classes).double()
    if model_name == "Res15":
        model = ResNet15_1D_Model(time_series_length=time_series_length, meta_data_size=meta_data_size,num_classes=num_classes).double()
    return model

def read_trained_model(snap_dir,model_name, acoustic_type, context_type,output_type,folds,max_epochs):
    context_combinations = []
    for r in range(len(context_type) + 1): 
        context_combinations.extend(itertools.combinations(context_type, r))
    train_acc_list = []
    test_acc_list = []
    fold_i_list = []
    inputs_list = []
    outputs_list = []
    model_list = []

    for _output in output_type:
        for _model in model_name:
            for acoustic in acoustic_type:
                for context in context_combinations:
                    if len(context)>0:
                        inputs = f"{acoustic}+{'+'.join(list(context))}"
                    else:
                        inputs = acoustic
                    
                    for fold in range(folds):
                        file_path = f"{_model}_classification_input_{inputs}_output_{_output}_roi_time10_roi_radius3_fold{fold}_of_folds10.pt"
                        file_path = os.path.join(snap_dir,file_path)
                        model = make_model(_model, inputs, _output)
    return model,file_path



In [8]:
data_loader = DataLoader(dataset, batch_size=8, shuffle=False)

In [13]:
meta_list = []
_cube_position, _laser_power, _scanning_speed, _regime_info, _print_direction, _mic, _ae, _defect_labels = next(iter(data_loader))
time_series = (transform_ft()(standardize_tensor(_mic))).double()
meta_list.append(_laser_power.double())
_ae.shape, time_series.shape

(torch.Size([8, 11760]), torch.Size([8, 5880]))

In [ ]:
model,file_path = read_trained_model(snap_dir,model_name,acoustic_type,context_type,output_type,folds,max_epochs)

In [ ]:
snapshot = torch.load(file_path, map_location=f"cuda:0", weights_only=True)
_state_dict = snapshot["model_state_dict"]
model.load_state_dict(_state_dict)
model = model.to('cuda')

In [ ]:
model(time_series.to('cuda'),meta_list)

In [ ]:
def make_model(input_type, output_type, time_series_length=5888):
    meta_data_size = len(input_type.split("+"))-1
    # time_series_length = 5888
    print(f"Input type: {input_type}")
    print(f"Output type: {output_type}")
    print(f"Lenght of time series data: {time_series_length}")
    print(f"Lenght of context data: {meta_data_size}")

    num_classes = 1
    if output_type == "regime":
        num_classes = 4
    if output_type == "defect":
        num_classes = 2
    if output_type == "direction":
        num_classes = 5
    if output_type == "position":
        num_classes = 5

    if model_name == "SVM":
        model = SVMModel(time_series_length+meta_data_size,num_classes=num_classes).double()
    if model_name == "CNN":
        model = CNN_Base_1D_Model(time_series_length=time_series_length, meta_data_size=meta_data_size,num_classes=num_classes).double()
    if model_name == "Res15":
        model = ResNet15_1D_Model(time_series_length=time_series_length, meta_data_size=meta_data_size,num_classes=num_classes).double()
    return model
# make_model(acoustic, _output)
# model.load_state_dict(_state_dict)